Test to chunk and make embeddings

In [15]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document

In [3]:
##searching if their is missing data in the csv
##missing 31 stories because riot still doesn't wrote it 
df = pd.read_csv("../data/raw/lore.csv")
print(df.isnull().sum())
null_data = df[df.isnull().any(axis=1)]
df.fillna("L'éditeur Riot Games n'a pas encore fourni l'histoire de ce champion", inplace = True)
#df
#null_data

Name          0
Region        0
SumUp         0
Biography     0
Story        31
dtype: int64


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=30,
    length_function=len,
    is_separator_regex=False,
)   

liste_chunk = []

for index, row in df.iterrows():
    text_biography = row['Biography']
    text_story = row['Story']
    texts_bio = text_splitter.create_documents([text_biography],[{"Source": "Biography", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    text_story = text_splitter.create_documents([text_story], [{"Source": "Story", "Name": row['Name'], "Region": row['Region'], "Sum_up": row['SumUp']}])
    liste_chunk.extend(texts_bio)
    liste_chunk.extend(text_story)


liste_chunk[1].__dict__


{'id': None,
 'metadata': {'Source': 'Biography',
  'Name': 'AATROX',
  'Region': 'RUNETERRA',
  'Sum_up': "Autrefois, Aatrox et ses frères étaient honorés pour avoir défendu Shurima contre le Néant. Mais ils finirent par devenir une menace plus grande encore pour Runeterra : la ruse et la sorcellerie furent employées pour les battre. Cependant, après des siècles d'emprisonnement, Aatrox fut le premier à retrouver sa liberté, en corrompant et transformant les mortels assez stupides pour tenter de s'emparer de l'arme magique qui contenait son essence. Désormais en possession d'un corps qu'il a approximativement transformé pour rappeler son ancienne forme, il arpente Runeterra en cherchant à assouvir sa vengeance apocalyptique."},
 'page_content': "Shurima fut conduit devant le Disque solaire pour devenir l'avatar d'une entité céleste aujourd'hui oubliée. Transfiguré, il obtint des ailes dorées comme la lumière de l'aube. Son armure étincelait comme une constellation d'espoir d'au-delà d

In [13]:
df = pd.read_csv("../data/processed/lore_chunked.csv", sep="\t")

In [14]:
embedding_model = SentenceTransformerEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    )
vectorstore = FAISS.load_local("../data/vectorstores/faiss_index", embedding_model, allow_dangerous_deserialization=True)
print("Nombre total de vecteurs :", vectorstore.index.ntotal)

C:\Users\bapti\AppData\Local\Temp\ipykernel_1392\590564233.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(
d:\Programmation\environnement_python\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Programmation\environnement_python\rag\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not suppo

Nombre total de vecteurs : 7792


We split the user_query and use rapid fuzz token ratio to filter on the champion

In [16]:
from rapidfuzz import process, fuzz

user_query = "C'est quoi l'histoire de atrox"  
splitted_input = user_query.split()
all_names = df["Name"].unique()

for element in splitted_input:
    best_match_name, score, _ = process.extractOne(element.upper(), all_names, scorer=fuzz.token_ratio)
    if score > 90:
        champion_detected = best_match_name

champion_detected


'AATROX'

In [17]:
chunks_for_champion = df[df["Name"] == champion_detected]

# Créer un vectorstore temporaire pour ce champion
vectorstore_filtered = FAISS.from_documents(
    [Document(page_content=row["page_content"], metadata=row.to_dict())
     for _, row in chunks_for_champion.iterrows()],
    embedding_model
)


In [18]:
results = vectorstore_filtered.similarity_search(user_query, k=10)
print(len(results))
for r in results:
    print(r.page_content[:200])  

10
porteur d'épée. D'un meilleur réceptacle. Autour de moi, il n'y a que des morts et des agonisants. J'entends leurs âmes se retirer de ce monde. Le combat n'a pas pris fin. Il continue dans l'enceinte 
déchus soient aussi dangereux pour la survie de Runeterra que les incursions du Néant, les Targoniens intervinrent. On prétend que la Manifestation du crépuscule donna aux mortels la connaissance néce
Qu'on la prenne pour un dieu ou pour un démon, l'Épée des Darkin a fait l'objet de nombreux contes... mais peu connaissent son nom véritable ou l'histoire de sa chute. Aux temps les plus reculés, avan
mes hôtes. Les ténèbres. Il pleuvait pendant la bataille. Et si la boue me recouvre ? Si je suis enfoui pendant des milliers d'années ? Enfermé dans cette prison ? L'horreur de cette perspective nourr
et la haine grandirent dans son cœur. Les puissances célestes qu'Aatrox avait autrefois incarnées avaient été balayées de ce monde et du souvenir des hommes. Furieux contre cette injustice, Aat

Concatenate all the result to build a context for the RAG

In [19]:
retrieved_chunk = "\n".join([r.page_content for r in results])

In [ ]:
import os
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()
api_key = os.environ.get("MISTRAL_API_KEY") ## get the api key using the .env hidden file
client = Mistral(api_key=api_key)

In [ ]:
prompt = f"""
Le contexte de la question est en dessous.
---------------------
{retrieved_chunk}
---------------------
Tu es un expert geek et connais tout l'histoire du jeu League of Legends. En t'appuyant sur le contexte la question et de la question posée, peux tu répondres de façon claire et concise.
Query: {user_query}
Answer:
"""
#prompt

In [ ]:
def run_mistral(user_message, model="mistral-large-latest"):

    messages = [
        {
            "role": "user", "content": user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return (chat_response.choices[0].message.content)

run_mistral(prompt)